In [2]:
from aimet_torch.quantsim import QuantizationSimModel
from aimet_common.defs import QuantScheme

In [ ]:
import os
import time
import datetime
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from torchvision import models, transforms
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.profilers import PyTorchProfiler
import sys

# Import your dataset modules
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(ROOT_DIR)

import dataset.DatasetReader as DatasetReader
import dataset.utils as utils

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations
transform = transforms.Compose([
    transforms.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p = 0.3),
    transforms.ColorJitter(brightness = 0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define dataset paths
root_folder = r"../../datasets/imagenet/new_imgnet_coco"
annotation_file = os.path.join(r"../../datasets", "coco", "annotations", "instances_train2017.json")
image_dir = os.path.join(r"../../datasets", "coco", "train2017")

# Load datasets
dataset_coco = DatasetReader.COCODataset(
    annotation_file=annotation_file,
    image_dir=image_dir,
    target_classes=[s.lower() for s in utils.GLOBAL_CLASSES],
    transform=transform,
    min_size = 60
)

class_names = [s.lower().replace(' ', '_') for s in utils.GLOBAL_CLASSES]
dataset_imagenet = DatasetReader.CustomImageFolder(root_dir=root_folder, class_names=class_names, transform=transform)

dataset = torch.utils.data.ConcatDataset([dataset_coco, dataset_imagenet])

# Define DataLoader
batch_size = 128*5
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=15, persistent_workers=True, pin_memory=True,prefetch_factor=2)

#Napravi cist AIMET Model bez kvantizacije
num_classes=64

base_model=models.mobilenet_v3_small(pretrained=False)
base_model.classifier[3]=nn.Linear(base_model.classifier[3].in_features,num_classes)

q_model=



Total annotations before filtering: 860001
Total annotations after filtering: 204543
Total COCO annotations: 860001
Filtered annotations: 204543
2025-02-28 13:47:10,743 - Quant - INFO - No config file provided, defaulting to config file at /home/centar15-desktop1/LPCV_2025_T1/.venv310/lib/python3.10/site-packages/aimet_common/quantsim_config/default_config.json
2025-02-28 13:47:10,753 - Quant - INFO - Unsupported op type Squeeze
2025-02-28 13:47:10,753 - Quant - INFO - Unsupported op type Mean
2025-02-28 13:47:10,761 - Quant - INFO - Selecting DefaultOpInstanceConfigGenerator to compute the specialized config. hw_version:default


RuntimeError: Encoding dictionary contains modules/parameters that doesn't exist in the model: /avgpool/GlobalAveragePool_output_0, /classifier/0/Gemm_output_0, /classifier/1/HardSwish_output_0, /features/0/0/Conv_output_0, /features/0/2/HardSwish_output_0, /features/1/block/0/2/Relu_output_0, /features/1/block/1/Mul_output_0, /features/1/block/1/activation/Relu_output_0, /features/1/block/1/avgpool/GlobalAveragePool_output_0, /features/1/block/1/fc2/Conv_output_0, /features/1/block/1/scale_activation/HardSigmoid_output_0, /features/1/block/2/0/Conv_output_0, /features/10/Add_output_0, /features/10/block/0/0/Conv_output_0, /features/10/block/0/2/HardSwish_output_0, /features/10/block/1/0/Conv_output_0, /features/10/block/1/2/HardSwish_output_0, /features/10/block/2/Mul_output_0, /features/10/block/2/activation/Relu_output_0, /features/10/block/2/avgpool/GlobalAveragePool_output_0, /features/10/block/2/fc2/Conv_output_0, /features/10/block/2/scale_activation/HardSigmoid_output_0, /featu

In [ ]:

# Kreira fake model, pa cemo mu ubaciti nas

dummy_input=torch.randn(1,3,224,224)
sim=QuantizationSimModel(
    model=q_model,
    dummy_input=dummy_input,
    quant_scheme=QuantScheme.training_range_learning_with_tf_enhanced_init,
    default_param_bw=8,
    default_output_bw=8

)
sim.load_encodings("quantized/mobilenetv3quantized.encodings")





class LightningQuantModel(pl.LightningModule):
    def __init__(self, model_for_qat, num_classes=64, learning_rate=2e-5,alpha=0.7):
        super(LightningQuantModel, self).__init__()
        self.model = model_for_qat
        self.criterion = nn.CrossEntropyLoss()
        self.learning_rate = learning_rate
        self.alpha = alpha

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, labels, ds = batch 
        images, labels = images.to(self.device), labels.to(self.device)
        outputs = self(images)
        loss = self.criterion(outputs, labels)

        ds = ds.to(self.device)
        alpha_mask = (ds == 0).float() * self.alpha + (ds == 1).float() * (1 - self.alpha)
        loss = loss * alpha_mask
        loss = loss.mean()

        # if(ds == 0):
        #     loss = self.alpha*loss
        # elif(ds == 1):
        #     loss = (1 - self.alpha)*loss
        
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.learning_rate)


# Define model checkpoint callback
checkpoint_callback = ModelCheckpoint(
    dirpath="models/",
    filename="quant_mobilenet_v3-{epoch:02d}-{train_loss:.2f}",
    save_top_k=3,
    monitor="train_loss",
    mode="min"
)

from pytorch_lightning.loggers import TensorBoardLogger

logger = TensorBoardLogger("logs/", name = "quant_mobilenet_v3", default_hp_metric=False)
profiler= PyTorchProfiler(on_trace_ready=torch.profiler.tensorboard_trace_handler("tb_logs/profiler0"),trace_memory=True,
                          schedule=torch.profiler.schedule(skip_first=10,warmup=1,wait=1,active=20))


quant_model = LightningQuantModel(sim.model)

trainer = Trainer(
    profiler=profiler,
    max_epochs=200,
    devices=1,  # Use only one GPU or CPU
    accelerator="auto",
    strategy="auto",  # Allows later multi-GPU setup without changing code
    precision = "16-mixed",
    callbacks=[checkpoint_callback],
    logger = logger
)

trainer.fit(quant_model, dataloader)

In [ ]:
import os
import time
import datetime
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from torchvision import models, transforms
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.profilers import PyTorchProfiler

# Import your dataset modules
from src.dataset.coco import get_coco
import src.dataset.DatasetReader as DatasetReader
import src.dataset.utils as utils

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations
transform = transforms.Compose([
    transforms.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p = 0.3),
    transforms.ColorJitter(brightness = 0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define dataset paths
root_folder = r"datasets/imagenet/new_imgnet_coco"
annotation_file = os.path.join(r"datasets", "coco", "annotations", "instances_train2017.json")
image_dir = os.path.join(r"datasets", "coco", "train2017")

# Load datasets
dataset_coco = DatasetReader.COCODataset(
    annotation_file=annotation_file,
    image_dir=image_dir,
    target_classes=[s.lower() for s in utils.GLOBAL_CLASSES],
    transform=transform,
    min_size = 60
)

class_names = [s.lower().replace(' ', '_') for s in utils.GLOBAL_CLASSES]
dataset_imagenet = DatasetReader.CustomImageFolder(root_dir=root_folder, class_names=class_names, transform=transform)

dataset = torch.utils.data.ConcatDataset([dataset_coco, dataset_imagenet])

# Define DataLoader
batch_size = 128*5
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=15, persistent_workers=True, pin_memory=True,prefetch_factor=2)

# Define Lightning Model
class LightningModel(pl.LightningModule):
    def __init__(self, num_classes=64, learning_rate=2e-5):
        super(LightningModel, self).__init__()
        self.model = models.mobilenet_v3_small(pretrained=True)
        self.model.classifier[3] = nn.Linear(self.model.classifier[3].in_features, num_classes)
        self.criterion = nn.CrossEntropyLoss()
        self.learning_rate = learning_rate
        self.alpha = 0.7

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, labels, ds = batch # ds = 0 - imagenet, ds = 1 - COCO
        images, labels = images.to(self.device), labels.to(self.device)
        outputs = self(images)
        loss = self.criterion(outputs, labels)

        ds = ds.to(self.device)
        alpha_mask = (ds == 0).float() * self.alpha + (ds == 1).float() * (1 - self.alpha)
        loss = loss * alpha_mask
        loss = loss.mean()

        # if(ds == 0):
        #     loss = self.alpha*loss
        # elif(ds == 1):
        #     loss = (1 - self.alpha)*loss
        
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.learning_rate)

# Define model checkpoint callback
checkpoint_callback = ModelCheckpoint(
    dirpath="models/",
    filename="mobilenet_v3-{epoch:02d}-{train_loss:.2f}",
    save_top_k=3,
    monitor="train_loss",
    mode="min"
)

from pytorch_lightning.loggers import TensorBoardLogger

logger = TensorBoardLogger("logs/", name = "mobilenet_v3", default_hp_metric=False)
profiler= PyTorchProfiler(on_trace_ready=torch.profiler.tensorboard_trace_handler("tb_logs/profiler0"),trace_memory=True,
                          schedule=torch.profiler.schedule(skip_first=10,warmup=1,wait=1,active=20))

# Initialize and train model using Lightning Trainer

model = LightningModel()

# model = LightningModel.load_from_checkpoint("models/mobilenet_v3-epoch=47-train_loss=0.18.ckpt")

trainer = Trainer(
    profiler=profiler,
    max_epochs=200,
    devices=1,  # Use only one GPU or CPU
    accelerator="auto",
    strategy="auto",  # Allows later multi-GPU setup without changing code
    precision = "16-mixed",
    callbacks=[checkpoint_callback],
    logger = logger
)

trainer.fit(model, dataloader)